In [65]:
import pandas as pd
# import nltk.data
from itertools import chain
from functools import reduce
import re
from math import log
from collections import Counter

In [7]:
# tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')

In [14]:
data = pd.read_json('Data/boletines-AMLO.json', dtype="dict", encoding="utf-8")
boletines = data['boletines']
corpus = [boletin["contenido"] for boletin in boletines]
counter_boletines = [Counter(contenido) for contenido in corpus_tokenizado]

In [35]:
corpus_tokenizado = [re.findall('[a-zA-zñáéíóúüÁÉÍÓÚÜÑ]+', contenido.lower()) for contenido in corpus if len(contenido)>2]
ch = chain(*corpus_tokenizado)
todos_los_tokens = list(ch)
df_corpus = pd.DataFrame(todos_los_tokens)
df_corpus.to_csv('Data/pln_corpus_amlo.csv')

In [36]:
with io.open('stop-words-spanish.txt', mode='r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()
corpus_tokenizado = [[w for w in doc if not (w in stopwords)] for doc in corpus_tokenizado]

In [83]:
def bm25(x, k=25):
    return ((x + 1)*k)/(x+k)

def tf(documento, func=bm25):
    c = Counter(documento)
    return {
        termino : bm25(cuenta) for (termino, cuenta) in c.items()
    }

def idf(palabra, documentos):
    return log(
        len(documentos) / 
        (1+reduce(lambda cuenta, documento : cuenta + (1 if (palabra in documento) else 0), documentos, 0))
    )

def tfidf(tf : dict, idfs : dict):
    return {
            palabra: idfs[palabra]*tf_value
            for (palabra, tf_value) in tf.items()
    }

def obtener_keywords(tfidfs, num_keywords=5):
    keywords = []
    for tfidf_n in tfidfs:
        keywords_n = sorted(tfidf_n.items() ,  key=lambda x: x[1], reverse=True)[:num_keywords]
        keywords.append([keyword[0] for keyword in keywords_n])
    return keywords

In [62]:
cuenta_total_palabras = reduce((lambda x, y: x + y), counter_boletines)
palabras = [palabra for (palabra, cuenta) in cuenta_total_palabras.items()]

In [67]:
idfs = {
    palabra: idf(palabra, corpus_tokenizado) for palabra in palabras
}

In [94]:
tfs = [tf(corpus_tokenizado[1]), tf(corpus_tokenizado[23])]

In [95]:
tfidfs = [
    {
        palabra: idfs[palabra]*tf
        for (palabra, tf) in tf_n.items()
    }
    for tf_n in tfs
]

In [96]:
keywords = obtener_keywords(tfidfs, 15)
keywords

[['catarina',
  'técnicos',
  'apodaca',
  'mesa',
  'santa',
  'tlc',
  'aeropuerto',
  'viable',
  'léon',
  'dosis',
  'alternativo',
  'sostienen',
  'hundimiento',
  'n',
  'tendrían'],
 ['moral',
  'populismo',
  'redes',
  'corral',
  'sucia',
  'ético',
  'chihuahua',
  'mensajes',
  'seguidores',
  'contracampaña',
  'imperativo',
  'dominando',
  'quisieron',
  'documental',
  'roberto']]